Dimer with Kanamori interaction
=======

We calculate the correlation functions 

$$\chi_{2,\sigma\sigma'}^{\rm PP} = \langle \left(c^\dagger_\sigma c^\dagger_{\sigma'}\right)(i\Omega) \left(c_\sigma c_{\sigma'}\right)(\tau=0) \rangle$$

and 

$$\chi_{2,\sigma\sigma'}^{\rm PH} = \langle n_\sigma (i\Omega) n_{\sigma'} (\tau=0) \rangle$$

of a dimer coupled with Kanamori interaction and a coupling to discrete bath states.

The non-interacting hamiltonian is most conveniently expressed using a matrix representation

$$
\hat{h} =
\begin{pmatrix}
\hat{h}_{0} & \hat{h}_{\rm coup} \\
\hat{h}_{\rm coup} & \hat{h}_{\rm bath}  \\
\end{pmatrix}=
\begin{pmatrix}
\begin{pmatrix}
\epsilon_0 -\mu & -t \\
-t & \epsilon_1 -\mu
\end{pmatrix} & 
\begin{pmatrix}
1 & 0 \\
0 & 1
\end{pmatrix}
\\
\begin{pmatrix}
1 & 0 \\
0 & 1
\end{pmatrix} & 
\begin{pmatrix}
\epsilon_{0,{\rm bath}} & -t_{\rm bath} \\
-t_{\rm bath} & \epsilon_{1,{\rm bath}}
\end{pmatrix}
\end{pmatrix}
$$

The impurity interaction is of the Kanamori-type, and reads

$$h_{\rm int} = \sum_{i,j} U n_{\uparrow,i} n_{\downarrow,j} + \sum_{\sigma,i\neq j} U' n_{\sigma,i} n_{\sigma,j}
            - \sum_{i \neq j} J c^\dagger_{i \uparrow} c_{i \downarrow} c^\dagger_{j \downarrow} c_{j \uparrow}
            + \sum_{i \neq j} J c^\dagger_{i \uparrow} c^\dagger_{i \downarrow} c_{j \downarrow} c_{j \uparrow}$$
   
The parameters are defined below.

In [ ]:
# %load model.py
import sys, os
sys.path.append(os.getcwd() + '/../common')
from util import *

from pytriqs.gf import Gf, MeshImFreq, iOmega_n, inverse
from pytriqs.operators import c, c_dag, n
from pytriqs.operators.util.hamiltonians import h_int_kanamori
from itertools import product
from numpy import matrix, array, diag

# ==== System Parameters ====
beta = 5.                       # Inverse temperature
mu = 0.25                       # Chemical potential
eps = array([0.0, 0.1])         # Impurity site energies
t = 1.                          # Hopping between impurity sites

eps_bath = array([0.2, 0.15])   # Bath site energies
t_bath = 0.1                    # Hopping between bath sites

U = 1.                          # Density-density interaction for opposite spins
Up = 0.3                        # Density-density interaction for equal spins
J = 0.5                         # Hunds coupling

spin_names = ['up', 'dn']
orb_names  = [0, 1]

# Non-interacting impurity hamiltonian in matrix representation
h_0_mat = diag(eps - mu) - matrix([[0, t],
                                   [t, 0]])

# Bath hamiltonian in matrix representation
h_bath_mat = diag(eps_bath) - matrix([[0, t_bath],
                                      [t_bath, 0]])

# Coupling matrix
V_mat = matrix([[1., 0.],
                [0., 1.]])

# ==== Local Hamiltonian ====
c_dag_vec = { s: matrix([[c_dag(s,o) for o in orb_names]]) for s in spin_names }
c_vec =     { s: matrix([[c(s,o)] for o in orb_names]) for s in spin_names }

h_0 = sum(c_dag_vec[s] * h_0_mat * c_vec[s] for s in spin_names)[0,0]

h_int = h_int_kanamori(spin_names, orb_names,
                        array([[0,Up-3*J],[Up-3*J,0]]), # Interaction for equal spins
                        array([[U,U-2*J],[U-2*J,U]]),   # Interaction for opposite spins
                        J,True)

h_loc = h_0 + h_int

# ==== Bath & Coupling hamiltonian ====
orb_bath_names = ['b_' + str(o) for o in orb_names]
c_dag_bath_vec = { s: matrix([[c_dag(s, o) for o in orb_bath_names]]) for s in spin_names }
c_bath_vec =     { s: matrix([[c(s, o)] for o in orb_bath_names]) for s in spin_names }

h_bath = sum(c_dag_bath_vec[s] * h_bath_mat * c_bath_vec[s] for s in spin_names)[0,0]
h_coup = sum(c_dag_vec[s] * V_mat * c_bath_vec[s] + c_dag_bath_vec[s] * V_mat * c_vec[s] for s in spin_names)[0,0] # FIXME Adjoint

# ==== Total impurity hamiltonian ====
h_imp = h_loc + h_coup + h_bath

# ==== Green function structure ====
gf_struct = [ [s, orb_names] for s in spin_names ]

# ==== Hybridization Function ====
n_iw = 10
iw_mesh = MeshImFreq(beta, 'Fermion', n_iw)
Delta = BlockGf(mesh=iw_mesh, gf_struct=gf_struct)
Delta << inverse(iOmega_n - V_mat * h_bath_mat * V_mat);

# ==== Non-Interacting Impurity Green function  ====
G0_iw = Delta.copy()
G0_iw['up'] << inverse(iOmega_n - h_0_mat - Delta['up']) # FIXME Should work for BlockGf
G0_iw['dn'] << inverse(iOmega_n - h_0_mat - Delta['dn'])


Results
====

In [ ]:
from pytriqs.plot.mpl_interface import oplot, plt
%matplotlib inline

In [ ]:
# %load ../common/plot.py
import sys, os
sys.path.append(os.getcwd() + "/..")
sys.path.append(os.getcwd() + "/../../common")
from model import *

from pytriqs.archive import HDFArchive
from pytriqs.plot.mpl_interface import oplot, plt
from glob import glob
from os.path import basename

# === Load chi2pp and chi2ph for every solver

solver_lst = [ basename(f).strip('.h5') for f in glob('results/*.h5') ]
chi2pp_tau, chi2ph_tau = {}, {}

for solver in solver_lst:
    dat = HDFArchive('results/' + solver + '.h5','r')
    chi2pp_tau[solver] = dat['chi2pp_tau']
    chi2ph_tau[solver] = dat['chi2ph_tau']
    
# === For every block and solver, plot chi2pp and chi2ph

block_lst = [bl for bl in chi2pp_tau[solver_lst[0]].indices]
n_blocks = len(block_lst)

for chi2, name in [[chi2pp_tau, r'$\chi_2^{\rm PP}$'], [chi2ph_tau, r'$\chi_2^{\rm PH}$']]:

    for block in block_lst:

        if max(norm_inf(chi2[solver][block][0,0,0,0]) for solver in solver_lst) < 1e-14: continue
        
        plt.figure(figsize=(10,6))
        for solver in solver_lst:
            oplot(chi2[solver][block][0,0,0,0], name = name + "_%s" % solver)

        plt.suptitle(name + "[" + str(block) + r"][0,0,0,0]", fontsize=14, y=1.05)
        plt.xlabel(r"$\tau$")

        plt.tight_layout()
        plt.show()


Deviations
======

We present a table containing deviations between the different solvers measured via

$$||G_{\rm Solver_1} - G_{\rm Solver_2}||_\infty$$

In [ ]:
for chi2, name in [[chi2pp_tau, 'chi_2^{PP}'], [chi2ph_tau, 'chi_2^{PH}']]:

    for block in block_lst:
        
        if max(norm_inf(chi2[solver][block]) for solver in solver_lst) < 1e-14: continue
        
        deviations = [[ norm_inf(chi2[s1][block].data - chi2[s2][block].data) for s1 in solver_lst ] \
                        for s2 in solver_lst ]
    
        print "\t\t    Deviations for " + name + str(block)
        print "\t\t -----------------------------------"

        row_format ="{:>15}" * (len(solver_lst) + 1)
        print row_format.format("", *solver_lst)
        row_format ="{:>15}" + "{:>15.2E}" * len(solver_lst)
        for solver, row in zip(solver_lst, deviations):
            print row_format.format(solver, *row)

        print "\n\n"